In [4]:
%%time
import xarray as xr
import numpy as np
import skimage.measure
import pandas as pd
from scipy.spatial import distance
import skimage.measure
from skimage.morphology import remove_small_objects,closing,binary_closing
from scipy import ndimage
from datetime import datetime,timedelta
import os
from dask.distributed import Client
import glob
from shapely.geometry import MultiPoint
import geopandas as gpd
import cartopy.crs as ccrs
import rioxarray as rxr

# Create a path to the code file
codeDir = os.path.dirname(os.path.abspath(os.getcwd()))
parentDir = os.path.dirname(codeDir)

precip_folder = os.path.join(parentDir,"precip_stats")

os.chdir(precip_folder)
filenames_coord = glob.glob('*coord')

# open aspect
a = '\\CO_SRTM1arcsec_aspect.tif'
folder = parentDir
elev = folder+a
codtm = rxr.open_rasterio(a)
# change lon to match global lat/lon in grib file
codtm = codtm.assign_coords(x=(((codtm.x + 360))))
codtm = codtm.rename({'x':'longitude','y':'latitude'})
codtm = codtm.sel(band=1)
codtm = codtm.where(codtm>=0)

for i in range(len(filenames_coord)):
    coord = pd.read_feather(precip_folder+'\\'+filenames_coord[i])
    aspect=[]

    for j in range(len(coord)):
        length = len(coord.latitude[j])
        fill = np.full(length,1)

        d = {'latitude':coord.latitude[j],'longitude':coord.longitude[j],'fill':fill}
        a = pd.DataFrame(data=d)

        a = a.groupby(['latitude','longitude']).mean()
        a = a.to_xarray()

        asp = codtm.sel(latitude=a.latitude,longitude=a.longitude,method='nearest')

        ds = xr.Dataset(data_vars=dict(asp=(["latitude", "longitude"], asp.values)),
            coords=dict(latitude=a.latitude,longitude=a.longitude,))

        a = a.assign({'aspect':ds.asp})
        a = a.where(a.fill==1)

        aspect.append(a.aspect.mean())
        
    aspect = [i.values for i in aspect]

    #save
    df = pd.DataFrame({'storm_id':coord['storm_id'],'aspect':aspect})
    
    df.aspect = df.aspect.astype('float')

    name = filenames_coord[i][0:8]+'_aspect'
    output = parentDir+'\\precip_stats\\'+name
    df.to_feather(output)


CPU times: total: 7h 47min 40s
Wall time: 7h 47min 42s
